In [39]:
import pandas
import numpy as np
import os
import shutil
from pathlib import Path
import json
import math
from PIL import Image, ImageDraw

In [40]:
def norm_label(s: str) -> str:
    return s.strip().casefold()

In [41]:
def labelme_json_to_mask(json_path: Path, image_root: Path, class_map: dict) -> np.ndarray:
    with open(json_path, "r") as f:
        anno = json.load(f)

    img_name = anno.get("imagePath", "")
    print(f"file={json_path.stem}, image={img_name}")
    W = anno.get("imageWidth"); H = anno.get("imageHeight")
    if (W is None or H is None) and img_name:
        with Image.open(image_root / img_name) as im:
            W, H = im.size
    if W is None or H is None:
        raise ValueError(f"Cannot determine size for {json_path}")
    if not (Path(img_name).stem).isdigit():
        print(f"Warning: imagePath is not numeric: {img_name}, file={json_path}")
    mask_img = Image.new("L", (W, H), color=int(class_map.get("background", 0)))
    draw = ImageDraw.Draw(mask_img)

    for s in anno.get("shapes", []):
        label = norm_label(s.get("label", ""))
        print(f"Label: {label}")
        cls_id = class_map.get(label)
        print(f"Class ID: {cls_id}")
        if cls_id is None:
            continue
        pts = [(float(x), float(y)) for x, y in s.get("points", [])]
        print(f"Points: {pts}")
        st = s.get("shape_type", "polygon")
        if st == "polygon" and len(pts) >= 3:
            draw.polygon(pts, outline=int(cls_id), fill=int(cls_id))
        elif st == "rectangle" and len(pts) >= 2:
            (x1, y1), (x2, y2) = pts[:2]
            draw.rectangle([x1, y1, x2, y2], outline=int(cls_id), fill=int(cls_id))
        elif st == "circle" and len(pts) >= 2:
            (cx, cy), (px, py) = pts[:2]
            r = math.hypot(px - cx, py - cy)
            draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=int(cls_id), fill=int(cls_id))
        print(f"Drew shape: {st} with class ID {cls_id}")
        
    return np.array(mask_img, dtype=np.uint8)

In [ ]:
data_dir = Path().cwd().parent.parent / "dataset"
if not data_dir.exists():
    raise FileNotFoundError(f"Data directory {data_dir} does not exist.")

images_dir = data_dir / "images"
if not images_dir.exists():
    raise FileNotFoundError(f"Images directory {images_dir} does not exist.")
labels_dir = data_dir / "labels"
if not labels_dir.exists():
    raise FileNotFoundError(f"Labels directory {labels_dir} does not exist.")
masks_dir = data_dir / "masks"
if not masks_dir.exists():

    raise FileNotFoundError(f"Masks directory {masks_dir} does not exist.")

# images_dir.mkdir(exist_ok=True)
# labels_dir.mkdir(exist_ok=True)
# masks_dir.mkdir(exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Images directory: {images_dir}")
print(f"Labels directory: {labels_dir}")

Data directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset
Images directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/images
Labels directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/labels


In [43]:
all_labels = set()

for jp in labels_dir.glob("*.json"):
    with open(jp, "r") as f:
        anno = json.load(f)
    for shape in anno.get("shapes", []):
        lbl = shape.get("label", "")
        all_labels.add(lbl)

print("Found labels:", all_labels)

Found labels: {'nerve', 'Nerve'}


In [44]:
CLASS_MAP = {"background": 0, "nerve": 1}

In [45]:

def safe_int_stem(p: Path):
    # allows sorting by number even if some names aren’t numeric
    return int(p.stem) if p.stem.isdigit() else float('inf')

for jp in sorted(labels_dir.glob("*.json"), key=safe_int_stem):
    with open(jp, "r") as f:
        meta = json.load(f)

    # Take only the stem and force .png output
    img_stem = Path(meta.get("imagePath", jp.with_suffix(".png").name)).stem
    out = masks_dir / f"{img_stem}.png"

    # Create mask from labelme JSON
    mask = labelme_json_to_mask(jp, images_dir, CLASS_MAP)

    # Binary mask → multiply by 255 for saving
    vis = (mask.astype(np.uint8) * 255)

    Image.fromarray(vis).save(out)
    # print("Saved:", out)

file=1, image=1.png
Label: nerve
Class ID: 1
Points: [(526.8412698412699, 396.6190476190476), (495.09523809523813, 386.3015873015873), (455.41269841269843, 420.42857142857144), (453.82539682539687, 446.6190476190476), (487.1587301587302, 452.1746031746032), (509.38095238095235, 436.3015873015873), (497.47619047619054, 426.77777777777777), (495.09523809523813, 420.42857142857144), (507.00000000000006, 409.3174603174603)]
Drew shape: polygon with class ID 1
file=2, image=../2.png
Label: nerve
Class ID: 1
Points: [(503.1009174311927, 406.5688073394495), (474.6605504587156, 405.65137614678895), (441.63302752293583, 421.2477064220183), (446.22018348623857, 432.2568807339449), (448.0550458715597, 443.26605504587155), (475.5779816513762, 447.8532110091743), (510.4403669724771, 449.6880733944954), (495.76146788990826, 440.5137614678899)]
Drew shape: polygon with class ID 1
file=3, image=3.png
Label: nerve
Class ID: 1
Points: [(534.2935779816514, 370.7889908256881), (513.1926605504586, 368.0366

In [46]:
img = Image.open(images_dir / img_name).convert("RGB")
mask = labelme_json_to_mask(labels_dir/ "1.json", images_dir, CLASS_MAP)

alpha = 0.45
rgb = np.array(img, np.float32)
m = (mask > 0).astype(np.float32)[..., None]
overlay = rgb*(1 - alpha*m) + np.array([255,0,0], np.float32)*alpha*m
Image.fromarray(overlay.astype(np.uint8)).save("overlay.png")

file=1, image=1.png
Label: nerve
Class ID: 1
Points: [(526.8412698412699, 396.6190476190476), (495.09523809523813, 386.3015873015873), (455.41269841269843, 420.42857142857144), (453.82539682539687, 446.6190476190476), (487.1587301587302, 452.1746031746032), (509.38095238095235, 436.3015873015873), (497.47619047619054, 426.77777777777777), (495.09523809523813, 420.42857142857144), (507.00000000000006, 409.3174603174603)]
Drew shape: polygon with class ID 1
